In [6]:
pip install PyMuPDF


   ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
   ------- -------------------------------- 3.4/18.5 MB 19.5 MB/s eta 0:00:01
   ----------------- ---------------------- 8.1/18.5 MB 19.9 MB/s eta 0:00:01
   -------------------------- ------------- 12.3/18.5 MB 20.1 MB/s eta 0:00:01
   ----------------------------------- ---- 16.5/18.5 MB 20.0 MB/s eta 0:00:01
   ---------------------------------------- 18.5/18.5 MB 18.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: c:\Users\cleme\.pyenv\pyenv-win\versions\3.13.1\python.exe -m pip install --upgrade pip


In [8]:
import os
from PIL import Image, ImageEnhance
import fitz  # PyMuPDF

def preprocess_images_for_ocr(dossier_entree, dossier_sortie,
                              facteur_luminosite=1.2,
                              facteur_contraste=1.5,
                              format_sortie='PNG',
                              dpi=300,
                              reconvertir_pdf=True):
    """
    Prétraite les images et PDF (via PyMuPDF) pour l'OCR : niveaux de gris, contraste, luminosité.
    Option de reconversion en un PDF unique par fichier.
    """
    for racine, _, fichiers in os.walk(dossier_entree):
        for fichier in fichiers:
            chemin_entree = os.path.join(racine, fichier)
            chemin_relatif = os.path.relpath(racine, dossier_entree)
            dossier_cible = os.path.join(dossier_sortie, chemin_relatif)
            os.makedirs(dossier_cible, exist_ok=True)

            images_finales = []  # Pour la fusion en PDF

            try:
                if fichier.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    images = [Image.open(chemin_entree)]
                elif fichier.lower().endswith('.pdf'):
                    images = []
                    with fitz.open(chemin_entree) as doc:
                        for page in doc:
                            pix = page.get_pixmap(dpi=dpi)
                            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
                            images.append(img)
                else:
                    continue  # Fichier non pris en charge

                for idx, img in enumerate(images):
                    img = img.convert('L')  # Niveaux de gris
                    img = ImageEnhance.Brightness(img).enhance(facteur_luminosite)
                    img = ImageEnhance.Contrast(img).enhance(facteur_contraste)

                    nom_base = os.path.splitext(fichier)[0]
                    suffixe_page = f"_page{idx+1}" if len(images) > 1 else ""
                    nom_sortie_img = f"{nom_base}{suffixe_page}.{format_sortie.lower()}"
                    chemin_sortie_img = os.path.join(dossier_cible, nom_sortie_img)

                    img.save(chemin_sortie_img, format=format_sortie)
                    print(f"✔️ prêt : {chemin_entree} → {chemin_sortie_img}")

                    # Pour la fusion finale
                    images_finales.append(img.convert('RGB'))

                # Fusionner en un seul PDF final
                if reconvertir_pdf and images_finales and fichier.lower().endswith('.pdf'):
                    chemin_sortie_pdf = os.path.join(dossier_cible, f"{nom_base}_traité.pdf")
                    images_finales[0].save(chemin_sortie_pdf, save_all=True, append_images=images_finales[1:])
                    print(f"📄 PDF fusionné enregistré : {chemin_sortie_pdf}")

            except Exception as e:
                print(f"❌ Erreur avec {chemin_entree} : {e}")

# Exemple d’utilisation
dossier_entree = r"C:\2. Stage\8. A convertir"
dossier_sortie = r"C:\2. Stage\9. Convert"
preprocess_images_for_ocr(dossier_entree, dossier_sortie,
                          facteur_luminosite=1.2,
                          facteur_contraste=1.5,
                          format_sortie='PNG',
                          reconvertir_pdf=True)


✔️ OCR prêt : C:\2. Stage\8. A convertir\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268.pdf → C:\2. Stage\9. Convert\.\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268_page1.png
✔️ OCR prêt : C:\2. Stage\8. A convertir\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268.pdf → C:\2. Stage\9. Convert\.\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268_page2.png
✔️ OCR prêt : C:\2. Stage\8. A convertir\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268.pdf → C:\2. Stage\9. Convert\.\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268_page3.png
✔️ OCR prêt : C:\2. Stage\8. A convertir\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268.pdf → C:\2. Stage\9. Convert\.\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268_page4.png
✔️ OCR prêt : C:\2. Stage\8. A convertir\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268.pdf → C:\2. Stage\9. Convert\.\Bordeaux -- 1717 -- Exports&Imports -- AD33_C_4268_page5.png
✔️ OCR prêt : C:\2. Stage\8. A convertir\Bordeaux -- 1717 -- Exports&Imports -- 